### This is a scratch notbook so I can:

1. Learn how to use psycopg2 to interface with a postgreSQL database and python
2. Set up my databases to prefrom basic data analysis on (baseball stats databases)




To start I am going to bring in my standard imports for the project

In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from configparser import ConfigParser
import psycopg2

# The custom functions i wrote to work with my postgreSQL
# database and pandas
from postgres_files.postgres_funcs import config


### Pre Processing draft kings data
